## Libraries

In [ ]:
!pip install python-bcb
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
from IPython.display import clear_output

## Shortcuts

In [2]:
vector0 = np.zeros
frame = pd.DataFrame
date_object = datetime.strptime
today = date.today().strftime("%Y-%m-%d")

## Functions

In [3]:
def info(start, end, init):
    print(f'start date: {start}')
    print(f'end date: {end}')
    print(f'capital: {init}')

## Inputs

In [4]:
limit_date = date_object('1995-01-01', "%Y-%m-%d")
today = date_object(today, "%Y-%m-%d")

In [5]:
start_date = date_object(input('Start date: '), "%Y-%m-%d")
if(start_date < limit_date or start_date >= today):
    while(start_date < limit_date or start_date >= today):
        start_date = date_object(input(f"The start date must be greater than 1995-01-01 and sooner than {today}: "), "%Y-%m-%d")
        
clear_output(wait=True)
print(f'Start date: {start_date}')

Start date: 2000-01-01 00:00:00


In [6]:
end_date = date_object(input('End date: '), "%Y-%m-%d")
if(end_date <= start_date or end_date >= today):
    while(end_date <= start_date or end_date >= today):
        end_date = date_object(input(f"The end date must be later than {start_date} and sooner than {today}: "), "%Y-%m-%d")

clear_output(wait=True)
print(f'End date: {end_date}')

End date: 2022-03-31 00:00:00


In [7]:
init_capital = float(input('Initial amount of capital: '))

Initial amount of capital: 1000


## Solution 

In [8]:
from bcb import sgs

taxa_selic = sgs.get({'Selic': 11})
df = taxa_selic.loc[start_date : end_date]
selic = df['Selic']

In [9]:
dcalendar_vector = pd.date_range(start= start_date, end= end_date)
mcalendar_vector = pd.date_range(start= start_date, end= end_date, freq='M')
ycalendar_vector = pd.date_range(start= start_date, end= end_date, freq='Y')

In [10]:
selic_ = selic.reindex(dcalendar_vector) 
selic_.fillna(value=0, inplace=True)

In [11]:
daily_earning = vector0(len(selic_))
capital = vector0(len(selic_))
total_earnings = vector0(len(selic_))
capital[0] = init_capital

In [12]:
for i in range(1,len(selic_)):
    daily_earning[i] = (capital[i-1])*(selic_[i-1]/100)
    capital[i] = capital[i-1]*(1+(selic_[i-1]/100))
    total_earnings[i] = total_earnings[i-1] + daily_earning[i]

In [13]:
df_date = frame({'Date': selic_.index}) #selic_ ta com frequencia diaria
df_capital = frame({'Capital': capital})
df_ae = frame({'Amount earned': total_earnings})

## Daily frequency

In [14]:
df_daily = pd.concat([df_date, df_capital, df_ae], axis=1)
df_daily.set_index("Date", inplace=True)
    
info(start_date, end_date, init_capital)
display(df_daily)

start date: 2000-01-01 00:00:00
end date: 2022-03-31 00:00:00
capital: 1000.0


,Capital,Amount earned
Date,,
2000-01-01,1000.000000,0.000000
2000-01-02,1000.000000,0.000000
2000-01-03,1000.000000,0.000000
2000-01-04,1000.691860,0.691860
2000-01-05,1001.384199,1.384199
...,...,...
2022-03-27,12506.524691,11506.524691
2022-03-28,12506.524691,11506.524691
2022-03-29,12511.994920,11511.994920


## Monthly frequency

In [15]:
df_freq = pd.concat([df_date, df_capital, df_ae], axis=1)
df_freq.set_index("Date", inplace=True)

df_index = frame({"Month Date": mcalendar_vector})
df_index.set_index("Month Date", inplace=True)

df_monthly = pd.merge(df_index, df_freq, left_index=True, right_index=True)
    
info(start_date, end_date, init_capital)
display(df_monthly)

start date: 2000-01-01 00:00:00
end date: 2022-03-31 00:00:00
capital: 1000.0


,Capital,Amount earned
2000-01-31,1013.860611,13.860611
2000-02-29,1028.571688,28.571688
2000-03-31,1043.490602,43.490602
2000-04-30,1057.723589,57.723589
2000-05-31,1072.803825,72.803825
...,...,...
2021-11-30,12133.859313,11133.859313
2021-12-31,12226.507366,11226.507366
2022-01-31,12316.038451,11316.038451
2022-02-28,12413.341603,11413.341603


## Yearly frequency

In [16]:
df_freq = pd.concat([df_date, df_capital, df_ae], axis=1)
df_freq.set_index("Date", inplace=True)

df_index = frame({"Year Date": ycalendar_vector})
df_index.set_index("Year Date", inplace=True)

df_yearly = pd.merge(df_index, df_freq, left_index=True, right_index=True)

info(start_date, end_date, init_capital)
display(df_yearly)

start date: 2000-01-01 00:00:00
end date: 2022-03-31 00:00:00
capital: 1000.0


,Capital,Amount earned
2000-12-31,1174.325422,174.325422
2001-12-31,1376.734949,376.734949
2002-12-31,1640.384862,640.384862
2003-12-31,2023.910928,1023.910928
2004-12-31,2352.608134,1352.608134
2005-12-31,2802.592125,1802.592125
2006-12-31,3225.133301,2225.133301
2007-12-31,3606.630937,2606.630937
2008-12-31,4056.432684,3056.432684
2009-12-31,4460.022551,3460.022551


## Most profitable period of n calendar days

In [17]:
n = int(input("Insert the number of calendar days: "))

Insert the number of calendar days: 200


In [18]:
df_selic = frame({'Selic': selic_})
df_selic_ = df_selic.reset_index()

In [19]:
retorno = vector0(len(df_selic_)-n)
cumulative_selic0 = 1

for i in range(0,n):
    cumulative_selic0 = ((cumulative_selic0)*(1+(df_selic_['Selic'][i])/100))

cumulative_selic0 = cumulative_selic0-1
retorno[0] = cumulative_selic0

In [20]:
for j in range (0,len(df_selic_)-(n+1)):
    retorno[j+1] = (retorno[j]/(1+(df_selic_['Selic'][j])))*(1+df_selic_['Selic'][j+n])

index_max = np.where(retorno == retorno.max())
index_max_profit = index_max[0][0]

## Answer 

In [21]:
profitable_start = dcalendar_vector[index_max_profit]
profitable_end = dcalendar_vector[index_max_profit + n]
print(f'Most profitable start date: {profitable_start}')
print(f'Most profitable end date: {profitable_end}')

Most profitable start date: 2003-01-06 00:00:00
Most profitable end date: 2003-07-25 00:00:00
